# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,15.46,90,100,4.92,NZ,1684516156
1,1,paciran,-6.8767,112.3761,24.75,86,91,2.91,ID,1684516156
2,2,port mathurin,-19.6833,63.4167,25.56,76,7,5.69,MU,1684516156
3,3,udachny,66.4167,112.4000,4.14,69,91,5.55,RU,1684516157
4,4,altai,46.3722,96.2583,-1.79,25,27,1.49,MN,1684516157


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.75,
    color = "City") 

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [21]:
# Narrow down cities that fit criteria and drop any results with null values
max_temp = 27
min_temp = 21
max_wind = 4.5
max_cloud = 0

city_data_df_filter = city_data_df[(city_data_df['Max Temp'] < max_temp) &
                                   (city_data_df['Max Temp'] > min_temp) &
                                   (city_data_df['Wind Speed'] < max_wind) &
                                   (city_data_df['Cloudiness'] == max_cloud)]

# Drop any rows with null values
city_data_df_filter = city_data_df_filter.dropna()

# Display sample data
city_data_df_filter


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
226,226,hailey,43.5196,-114.3153,21.05,37,0,2.57,US,1684516208
281,281,pendleton,45.6721,-118.7886,26.64,58,0,1.54,US,1684516220
375,375,central point,42.3760,-122.9164,23.69,59,0,1.54,US,1684516247
479,479,argostolion,38.1811,20.4890,21.04,73,0,0.00,GR,1684516271
493,493,abaete,-19.1600,-45.4458,26.81,29,0,1.61,BR,1684516275
501,501,farafangana,-22.8167,47.8333,23.10,85,0,2.54,MG,1684516276


### Step 3: Create a new DataFrame called `hotel_df`.

In [22]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df_filter[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
226,hailey,US,43.5196,-114.3153,37,
281,pendleton,US,45.6721,-118.7886,58,
375,central point,US,42.3760,-122.9164,59,
479,argostolion,GR,38.1811,20.4890,73,
493,abaete,BR,-19.1600,-45.4458,29,
501,farafangana,MG,-22.8167,47.8333,85,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [26]:
# Set parameters to search for a hotel
radius = 10000
limit = 10
params = {
    "limit":limit,
    "apiKey":geoapify_key   
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index,'Lat']
    longitude = hotel_df.loc[index,'Lng']

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    params["categories"] = f"accommodation.hotel"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hailey - nearest hotel: Fairfield Inn
pendleton - nearest hotel: Working Girls Hotel
central point - nearest hotel: La Quinta Inn & Suites by Wyndham Central Point - Medford
argostolion - nearest hotel: Ainos Hotel
abaete - nearest hotel: Mark's Hotel
farafangana - nearest hotel: Le Concombre Masqué


,City,Country,Lat,Lng,Humidity,Hotel Name
226,hailey,US,43.5196,-114.3153,37,Fairfield Inn
281,pendleton,US,45.6721,-118.7886,58,Working Girls Hotel
375,central point,US,42.3760,-122.9164,59,La Quinta Inn & Suites by Wyndham Central Poin...
479,argostolion,GR,38.1811,20.4890,73,Ainos Hotel
493,abaete,BR,-19.1600,-45.4458,29,Mark's Hotel
501,farafangana,MG,-22.8167,47.8333,85,Le Concombre Masqué


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [29]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 3,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]) 

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)